# Snowfall data
- Input data

In [59]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json

In [60]:
headers = {
    'Access-Control-Allow-Origin': '*',
    'Access-Control-Allow-Methods': 'GET',
    'Access-Control-Allow-Headers': 'Content-Type',
    'Access-Control-Max-Age': '3600',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }

## Get Ski Report

In [3]:
def get_ski_resorts(state_name):
    s = str.lower(state_name)
    url = f"https://www.onthesnow.com/{s}/skireport"
    req = requests.get(url, headers)
    soup = BeautifulSoup(req.content, 'html.parser')
    temp_elem_str = str(soup.find(id='__NEXT_DATA__')).partition('"data":')[2]    
    temp_elem_str = temp_elem_str[:temp_elem_str.find(']')]+']'
    _json = json.loads(temp_elem_str)
    return pd.json_normalize(_json)  

df_ca = get_ski_resorts('California')
df_co = get_ski_resorts('Colorado')
df_ut = get_ski_resorts('Utah')

df_resorts = pd.concat([df_ca, df_co, df_ut])


uuid                                                43
title                                    Bear Mountain
title_short                              Bear Mountain
title_original                           Bear Mountain
slug                                     bear-mountain
region                                      california
region_id                                          311
region_en                                   California
country_code                                       USA
domain                                            None
surfaceType                                          0
updatedAt                    2023-04-16T17:45:08+00:00
snow.last24                                        0.0
snow.base                                        190.5
snow.middle                                        NaN
snow.summit                                      254.0
snow.last72                                        0.0
lifts.open                                           7
lifts.tota

In [ ]:
df_resorts.head()

## Get Resort's Snow Report

In [57]:
def get_resort_snowfall(my_series, time_period="Annual", headers=headers):
    req = requests.get("https://www.onthesnow.com/{}/historical-snowfall".format(df_resorts.iloc[0].region+ '/' + df_resorts.iloc[0].slug), headers)
    soup = BeautifulSoup(req.content, 'html.parser')
    temp_elem_str = str(soup.find(id='__NEXT_DATA__')).partition(f'"snowfallInfo{time_period}":[')[2]    
    temp_elem_str = temp_elem_str[:temp_elem_str.find(']')]
    _json = json.loads('['+temp_elem_str+']')
    temp_df = pd.json_normalize(_json)  
    temp_df['time_period'] = time_period
    temp_df['title'] = my_series.title
    temp_df['region'] = my_series.region
    temp_df['slug'] = my_series.slug
    return temp_df

df_snowfall_annual = pd.DataFrame()
for i in range(0,len(df_resorts)):
    df_snowfall_annual = pd.concat([df_snowfall_annual, get_resort_snowfall(df_resorts.iloc[i])])

df_snowfall_monthly = pd.DataFrame()
for i in range(0,len(df_resorts)):
    df_snowfall_monthly = pd.concat([df_snowfall_monthly, get_resort_snowfall(df_resorts.iloc[i], time_period='Monthly')])


,date,totalSnow,snowDays,baseDepth,summitDepth,maxBaseDepth,biggestSnowfall,time_period,title,region,slug
0,2023-11-03,12.700000,1,26.636645,38.282128,76.20,45.72,Monthly,Bear Mountain,california,bear-mountain
1,2023-12-03,41.794545,3,48.715000,75.616235,91.44,45.72,Monthly,Bear Mountain,california,bear-mountain
2,2023-01-03,74.121818,4,70.572027,99.827295,116.84,71.12,Monthly,Bear Mountain,california,bear-mountain
3,2023-02-03,71.581818,4,84.143100,119.000440,190.50,71.12,Monthly,Bear Mountain,california,bear-mountain
4,2023-03-03,58.420000,3,84.914782,117.762988,254.00,76.20,Monthly,Bear Mountain,california,bear-mountain
...,...,...,...,...,...,...,...,...,...,...,...
3,2023-02-03,71.581818,4,84.143100,119.000440,190.50,71.12,Monthly,Solitude Mountain Resort,utah,solitude-mountain-resort
4,2023-03-03,58.420000,3,84.914782,117.762988,254.00,76.20,Monthly,Solitude Mountain Resort,utah,solitude-mountain-resort
5,2023-04-03,0.000000,0,48.279791,64.423636,190.50,0.00,Monthly,Solitude Mountain Resort,utah,solitude-mountain-resort
6,2023-05-03,0.000000,0,0.000000,0.000000,0.00,0.00,Monthly,Solitude Mountain Resort,utah,solitude-mountain-resort


In [65]:
pd.concat([df_snowfall_annual, df_snowfall_monthly]).to_csv('snowfall_output.csv')